### ИНТЕРАКТИВНЫЙ ДАШБОРД

In [ ]:
# импорт необходимых библиотек
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import numpy as np

In [ ]:
# ЗАГРУЗКА И ПРЕДОБРАБОТКА ДАННЫХ
df = pd.read_csv('../data/marketing_campaign.csv', sep='\t')

# Нормализация категорий
df["Education"] = df["Education"].replace({"2n Cycle": "Pre-Graduate", "Basic": "Pre-Graduate"})
df["Marital_Status"] = df["Marital_Status"].replace({
    "Married": "Married/Together", "Together": "Married/Together",
    "Single": "Single", "Divorced": "Other", "Widow": "Other",
    "Alone": "Other", "Absurd": "Other", "YOLO": "Other"
})

# Подготовка данных
df['Age'] = 2024 - df['Year_Birth']  
df['Total_Children'] = df['Kidhome'] + df['Teenhome']  
df['Total_Spending'] = df[['MntWines', 'MntFruits', 'MntMeatProducts', 
                           'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']].sum(axis=1) 

# Очистка Income
df['Income'] = df['Income'].fillna(df['Income'].median())
df = df[df['Income'] < 600000]

# Создаем признак "Есть дети" для фильтра
df['Has_Children'] = (df['Total_Children'] > 0).astype(str)
df['Has_Children'] = df['Has_Children'].replace({'True': 'Есть дети', 'False': 'Нет детей'})

In [ ]:
# 3 ФИЛЬТРА
# Фильтр 1: Образование 
education_filter = widgets.SelectMultiple(
    options=sorted(df['Education'].unique()),
    value=list(df['Education'].unique()),
    description='Образование:',
    disabled=False,
    layout=widgets.Layout(width='350px', height='100px')
)
# Фильтр 2: Семейное положение 
marital_filter = widgets.SelectMultiple(
    options=sorted(df['Marital_Status'].unique()),
    value=list(df['Marital_Status'].unique()),
    description='Семья:',
    disabled=False,
    layout=widgets.Layout(width='350px', height='100px')
)

# Фильтр 3: Наличие детей
children_filter = widgets.RadioButtons(
    options=['Все', 'Есть дети', 'Нет детей'],
    value='Все',
    description='Дети:',
    disabled=False,
    layout=widgets.Layout(width='200px')
)

# Кнопка обновления
update_button = widgets.Button(
    description='Обновить дашборд',
    button_style='success',
    icon='check',
    layout=widgets.Layout(width='200px', height='40px')
)

# Вывод для дашборда
output = widgets.Output()
